In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial import ConvexHull

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)
torch.set_printoptions(precision=12)

In [ ]:
data = pickle.load(open("../../data/basipetal.pkl", 'rb'))

Dx = 0.
Dy = 0.
height_source = 38.
height_target = 100.

source = torch.tensor(data['source_d'], dtype=torch.get_default_dtype())
target = torch.tensor(data['target_d'], dtype=torch.get_default_dtype())
source_curve = torch.tensor(data['source_c'], dtype=torch.get_default_dtype())
target_curve = torch.tensor(data['target_c'], dtype=torch.get_default_dtype())

smin, smax = torch.min(source_curve[:, 1]), torch.max(source_curve[:, 1])
sscale = height_source / (smax - smin)
source[:, 1] = Dy - sscale * (source[:, 1] - smax)
source[:, 0] = Dx + sscale * (source[:, 0] - torch.mean(source_curve[:, 0]))
source_curve[:, 1] = Dy - sscale * (source_curve[:, 1] - smax)
source_curve[:, 0] = Dx + sscale * (source_curve[:, 0] - torch.mean(source_curve[:, 0]))

tmin, tmax = torch.min(target_curve[:, 1]), torch.max(target_curve[:, 1])
tscale = height_target / (tmax - tmin)
target[:, 1] = - tscale * (target[:, 1] - tmax)
target[:, 0] = tscale * (target[:, 0] - torch.mean(target_curve[:, 0]))
target_curve[:, 1] = - tscale * (target_curve[:, 1] - tmax)
target_curve[:, 0] = tscale * (target_curve[:, 0] - torch.mean(target_curve[:, 0]))

source_curve_fit = source_curve.clone()
source_curve_fit = source_curve_fit[source_curve_fit[:, 0] <= 0]
target_curve_fit = target_curve.clone()
target_curve_fit = target_curve_fit[target_curve_fit[:, 0] <= 0]

hull_fit = ConvexHull(source_curve_fit)
source_curve_fit_convex = source_curve_fit[hull_fit.vertices]
hull = ConvexHull(source_curve)
source_curve_convex = source_curve[hull.vertices]

aabb = dm.Utilities.AABB.build_from_points(torch.cat([target_curve_fit, source_curve_fit]))
aabb_source_fit = dm.Utilities.AABB.build_from_points(source_curve_fit)
aabb_source = dm.Utilities.AABB.build_from_points(source_curve)
aabb.squared()

In [ ]:
pts_implicit1_step = 1.5
pts_implicit1_x, pts_implicit1_y = torch.meshgrid([
    torch.arange(0., aabb_source_fit.xmin-pts_implicit1_step, step=-pts_implicit1_step),
    torch.arange(aabb_source_fit.ymin, aabb_source_fit.ymax+pts_implicit1_step, step=pts_implicit1_step)])

pts_implicit1 = dm.Utilities.grid2vec(pts_implicit1_x, pts_implicit1_y)
pts_implicit1_mask = dm.Utilities.area_shape(pts_implicit1, shape=source_curve_fit)
pts_implicit1 = pts_implicit1[pts_implicit1_mask]

In [ ]:
%matplotlib qt5

plt.axis('equal')

plt.subplot(1, 2, 1)
plt.title("Source")
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.plot(source_curve_fit[:, 0].numpy(), source_curve_fit[:, 1].numpy(), '-')
plt.plot(source[:, 0].numpy(), source[:, 1].numpy(), 'x')
plt.plot(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), '.')
plt.axis('equal')

plt.subplot(1, 2, 2)
plt.title("Target")
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.plot(target_curve_fit[:, 0].numpy(), target_curve_fit[:, 1].numpy(), '-')
plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), 'x')
plt.axis('equal')

plt.show()

In [ ]:
sigma1 = 25.
nu1 = 0.001
coeff_global_trans = 1.
coeff1 = 1.
C = torch.ones(pts_implicit1.shape[0], 2, 1, requires_grad=True)
th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(t) for t in th])
implicit1 = dm.DeformationModules.ImplicitModule1(2, pts_implicit1.shape[0], sigma1, C, nu1, coeff1, gd=(pts_implicit1.clone().requires_grad_(), R.clone().requires_grad_()))
global_trans = dm.DeformationModules.GlobalTranslation(2, coeff=coeff_global_trans)

In [ ]:
# Setting up the model and start the fitting loop
model = dm.Models.ModelPointsRegistration(
    [source_curve_fit.clone(), source.clone()],
    [implicit1, global_trans],
    [dm.Attachment.VarifoldAttachment(2, [10., 50.], 0.1),
     dm.Attachment.EuclideanPointwiseDistanceAttachment(50.)],
    other_parameters=[implicit1.C], lam=1000.)

In [ ]:
fitter = dm.Models.ModelFittingScipy(model, 1.)

costs = fitter.fit([target_curve_fit, target], 500, options={}, disp=True)

In [ ]:
%matplotlib qt5
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.plot(range(len(costs)), costs)
plt.show()

In [ ]:
%matplotlib qt5
C = model.modules[2].C.detach()
R = model.modules[2].manifold.gd[1].detach()

points = model.modules[1].manifold.gd.detach().view(-1, 2)
out_curve = model.modules[0].manifold.gd.detach().view(-1, 2)
ax = plt.subplot(111, aspect='equal')
plt.axis('equal')
plt.plot(source[:, 0].numpy(), source[:, 1].numpy(), '.')
plt.plot(points[:, 0].numpy(), points[:, 1].numpy(), 'o')
plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), 'x')
plt.plot(target_curve_fit[:, 0].numpy(), target_curve_fit[:, 1].numpy(), '--')
plt.plot(out_curve[:, 0].numpy(), out_curve[:, 1].numpy(), '--')
for i in range(points.shape[0]):
   plt.plot([target[i, 0], points[i, 0]], [target[i, 1], points[i, 1]], '-')

def_grids_c = model.compute_deformation_grid([aabb_source.xmin, aabb_source.ymin],
                                             [aabb_source.width/2, aabb_source.height],
                                             [8, 16])

def_grid_c_x, def_grid_c_y = def_grids_c[0], def_grids_c[1]
dm.Utilities.plot_grid(ax, def_grid_c_x.numpy(), def_grid_c_y.numpy(), color='C0')

#dm.Utilities.plot_C_arrow(ax, source, C, R=R, scale=1., zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, R=R, scale=1., color='black')

plt.xlabel("$x$")
plt.ylabel("$y$")

plt.show()

In [ ]:
#source_mirrored = pts_implicit1.detach().clone()
#source_mirrored[:, 0] = -source_mirrored[:, 0]
# We now use the learned C to fit the whole leaf.
# We suppose C is symmetric around the vertical axis.

# 
source_mirrored = torch.cat([pts_implicit1, torch.tensor([-1., 1])*pts_implicit1])
C_mirrored = torch.cat([C.detach(), C.detach()])
R_mirrored = torch.cat([R, R])

pts_implicit0_small_scale = source_curve.detach().clone()
m_scale = 5.4
pts_implicit0_medium_scale_x, pts_implicit0_medium_scale_y = torch.meshgrid([
    torch.arange(aabb_source.xmin, aabb_source.xmax, step=m_scale),
    torch.arange(aabb_source.ymin, aabb_source.ymax, step=m_scale)])
pts_implicit0_medium_scale = dm.Utilities.grid2vec(pts_implicit0_medium_scale_x, pts_implicit0_medium_scale_y)
pts_implicit0_medium_scale = pts_implicit0_medium_scale[dm.Utilities.is_inside_shape(source_curve_convex, pts_implicit0_medium_scale)]

In [ ]:
plt.axis('equal')
plt.plot(source_curve[:, 0].numpy(), source_curve[:, 1].numpy())
plt.plot(source_mirrored[:, 0].numpy(), source_mirrored[:, 1].numpy(), '.')
plt.plot(pts_implicit0_small_scale[:, 0].numpy(), pts_implicit0_small_scale[:, 1].numpy(), 'x')
plt.plot(pts_implicit0_medium_scale[:, 0].numpy(), pts_implicit0_medium_scale[:, 1].numpy(), 'o')
plt.show()

In [ ]:
nu0 = 0.001
coeff0 = 5.
scale_small = 1.

implicit0_small = dm.DeformationModules.ImplicitModule0(dm.Manifolds.Landmarks(2, pts_implicit0_small_scale.shape[0], gd=pts_implicit0_small_scale.view(-1).requires_grad_()), scale_small, nu0, coeff0)
implicit0_medium = dm.DeformationModules.ImplicitModule0(dm.Manifolds.Landmarks(2, pts_implicit0_medium_scale.shape[0], gd=pts_implicit0_medium_scale.view(-1).requires_grad_()), m_scale, nu0, coeff0)
implicit1_mirrored = dm.DeformationModules.ImplicitModule1(dm.Manifolds.Stiefel(2, source_mirrored.shape[0], gd=(source_mirrored.view(-1).requires_grad_(), R_mirrored.view(-1).requires_grad_())), C_mirrored, sigma1, nu1, coeff1)
global_trans = dm.DeformationModules.GlobalTranslation(2, coeff=coeff_global_trans)

In [ ]:
# Setting up the model and start the fitting loop
model_mirrored = dm.Models.ModelPointsRegistration((source_curve,), [implicit1_mirrored, implicit0_small, implicit0_medium, global_trans], (dm.Attachment.VarifoldAttachement([10., 50.]),))

In [ ]:
fitter_mirrored = dm.Models.ModelFittingScipy(model_mirrored, 1., 1.)

costs_mirrored = fitter_mirrored.fit((target_curve,), 500, log_interval=10)

In [ ]:
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.plot(range(len(costs_mirrored)), costs_mirrored)
plt.show()

In [ ]:
# Some plots
%matplotlib qt5

fit_curve = model_mirrored.modules[0].manifold.gd.detach().view(-1, 2)
imp_points = model_mirrored.modules[1].manifold.gd[0].detach().view(-1, 2)

aabb = dm.Utilities.AABB.build_from_points(torch.cat([target_curve, fit_curve]))
def_grids = model_mirrored.compute_deformation_grid(torch.tensor([aabb_source.xmin, aabb_source.ymin]),
                                             torch.tensor([aabb_source.width, aabb_source.height]),
                                             torch.tensor([32, 32]))

ax = plt.subplot(111, aspect='equal')
plt.axis('equal')
plt.title('Source')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(source_mirrored[:, 0].numpy(), source_mirrored[:, 1].numpy(), '.')
dm.Utilities.plot_grid(ax, def_grids[-1][0].numpy(), def_grids[-1][1].numpy(), color='C0')
plt.plot(source_curve[:, 0].numpy(), source_curve[:, 1].numpy(), '-')
plt.plot(target_curve[:, 0].numpy(), target_curve[:, 1].numpy(), '-')
plt.plot(fit_curve[:, 0].numpy(), fit_curve[:, 1].numpy(), '--')
plt.plot(imp_points[:, 0].numpy(), imp_points[:, 1].numpy(), 'x')

plt.show()

In [ ]:
# We apply some deformation on the leaf shape
# Some additive gaussian noise
torch.manual_seed(1337)
target_curve_deformed = target_curve + 0.8*torch.randn_like(target_curve)
# Small translation
target_curve_deformed = target_curve_deformed + torch.tensor([5., 5.]).repeat([target_curve.shape[0], 1])
# Small rotation
r = dm.Utilities.rot2d(math.pi/16.)
for i in range(target_curve_deformed.shape[0]):
    target_curve_deformed[i] = torch.mm(target_curve_deformed[i].unsqueeze(0), r)

aabb_deformed = dm.Utilities.AABB.build_from_points(target_curve_deformed)

In [ ]:
plt.title("Deformed target")
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.axis('equal')
plt.plot(target_curve_deformed[:, 0].numpy(), target_curve_deformed[:, 1].numpy())
plt.show()

In [ ]:
implicit0_small = dm.DeformationModules.ImplicitModule0(dm.Manifolds.Landmarks(2, pts_implicit0_small_scale.shape[0], gd=pts_implicit0_small_scale.view(-1).requires_grad_()), 1., 0.001, 5.)
implicit0_medium = dm.DeformationModules.ImplicitModule0(dm.Manifolds.Landmarks(2, pts_implicit0_medium_scale.shape[0], gd=pts_implicit0_medium_scale.view(-1).requires_grad_()), m_scale, 0.001, 5.)
implicit1_mirrored = dm.DeformationModules.ImplicitModule1(dm.Manifolds.Stiefel(2, source_mirrored.shape[0], gd=(source_mirrored.view(-1).requires_grad_(), R_mirrored.view(-1).requires_grad_())), C_mirrored, sigma1, nu1, coeff1)
global_trans = dm.DeformationModules.GlobalTranslation(2, coeff=coeff_global_trans)

In [ ]:
# Setting up the model and start the fitting loop
model_deformed = dm.Models.ModelPointsRegistration((source_curve,), [implicit1_mirrored, implicit0_small, implicit0_medium, global_trans], (dm.Attachment.VarifoldAttachement([10., 50.]),))

fitter_deformed = dm.Models.ModelFittingScipy(model_deformed, 1., 1.)

In [ ]:
costs_deformed = fitter_deformed.fit((target_curve_deformed,), 500, log_interval=1)

In [ ]:
plt.plot(range(len(costs_deformed)), costs_deformed)
plt.show()

In [ ]:
# Some plots
%matplotlib qt5

fit_curve = model_deformed.modules[0].manifold.gd.detach().view(-1, 2)

aabb = dm.Utilities.AABB.build_from_points(torch.cat([target_curve, fit_curve]))
def_grids_deformed = model_deformed.compute_deformation_grid(torch.tensor([aabb_source.xmin, aabb_source.ymin]),
                                                             torch.tensor([aabb_source.width, aabb_source.height]),
                                                             torch.tensor([32, 32]))

ax = plt.subplot(111, aspect='equal')
plt.axis('equal')
plt.xlabel("$x$")
plt.ylabel("$y$")
dm.Utilities.plot_grid(ax, def_grids_deformed[-1][0].numpy(), def_grids_deformed[-1][1].numpy(), color='C0')
plt.plot(source_curve[:, 0].numpy(), source_curve[:, 1].numpy(), '-')
plt.plot(target_curve_deformed[:, 0].numpy(), target_curve_deformed[:, 1].numpy(), '-')
plt.plot(fit_curve[:, 0].numpy(), fit_curve[:, 1].numpy(), '--')

plt.show()